In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import csv
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Import API key
from api_keys import api_key

In [ ]:
# Loading files used in this analysis
tracts_df = pd.read_csv('500_Census_Tracts.csv')
acs16_df = pd.read_csv('ACS_2016.csv')
tracts_df.head()

## Selecting a smaller set of variables from the 2016 Census Tracts CDC dataset

In [ ]:
tracts_reduced = tracts_df[['StateAbbr', 'PlaceName', 'MHLTH_CrudePrev', 'PHLTH_CrudePrev', 'SLEEP_CrudePrev',
                            'LPA_CrudePrev', 'OBESITY_CrudePrev', 'CSMOKING_CrudePrev', 'BINGE_CrudePrev',
                            'ACCESS2_CrudePrev', 'CHECKUP_CrudePrev', 'DENTAL_CrudePrev', 'COREM_CrudePrev',
                            'COREW_CrudePrev', 'PlaceFIPS', 'TractFIPS', 'Place_TractID', 'Population2010',
                            'Geolocation']]
tracts_reduced.rename(columns = {
    'StateAbbr': 'state',
    'PlaceName': 'city',
    'PlaceFIPS': 'city_FIPS',
    'TractFIPS': 'tract_FIPS',
    'Place_TractID': 'place_tractID',
    'Population2010': 'population_2010',
    'ACCESS2_CrudePrev': 'PS_lack_health_ins',
    'BINGE_CrudePrev': 'UB_binge_drinking',
    'CHECKUP_CrudePrev': 'PS_routine_checkups',
    'COREM_CrudePrev': 'PS_older_men_uptodate',
    'COREW_CrudePrev': 'PS_older_women_uptodate',
    'CSMOKING_CrudePrev': 'UB_current_smoking',
    'DENTAL_CrudePrev': 'PS_dental_checkups',
    'LPA_CrudePrev': 'UB_lack_physical_activity',
    'MHLTH_CrudePrev': 'HO_poor_mental_health',
    'OBESITY_CrudePrev': 'UB_obesity',
    'PHLTH_CrudePrev': 'HO_poor_physical_health',
    'SLEEP_CrudePrev': 'UB_less_sleep',
    'Geolocation': 'geolocation'}, inplace = True)
tracts_reduced.head()

In [ ]:
tracts_reduced['geo_id'] = tracts_reduced.city_FIPS.apply(
    lambda x: '0' + str(x) if len(str(x)) == 6 else str(x))
len(tracts_reduced)

## Selecting a smaller set of variables from the 2016 American Community Survey (ACS) custom-built dataset

In [ ]:
acs16_reduced = acs16_df[['HC01_EST_VC01', 'HC03_EST_VC01', 'HC01_EST_VC04', 'HC01_EST_VC07', 'HC03_EST_VC07',
                          'HC01_EST_VC08', 'HC03_EST_VC08', 'HC01_EST_VC09', 'HC03_EST_VC09', 'HC01_EST_VC10',
                          'HC03_EST_VC10', 'HC01_EST_VC11', 'HC03_EST_VC11', 'HC01_EST_VC12', 'HC03_EST_VC12',
                          'HC01_EST_VC13', 'HC01_EST_VC20', 'HC03_EST_VC20', 'HC01_EST_VC21', 'HC03_EST_VC21',
                          'HC01_EST_VC32', 'HC03_EST_VC32', 'HC01_EST_VC33', 'HC03_EST_VC33',
                          'HC01_EST_VC50', 'HC03_EST_VC50', 'HC01_EST_VC54', 'HC03_EST_VC54', 'HC01_EST_VC55',
                          'HC03_EST_VC55', 'HC01_EST_VC56', 'HC03_EST_VC56', 'HC01_EST_VC57', 'HC03_EST_VC57',
                          'HC01_EST_VC58', 'HC03_EST_VC58', 'HC01_EST_VC74', 'HC03_EST_VC74', 'HC01_EST_VC75',
                          'HC03_EST_VC75', 'HC01_EST_VC76', 'HC03_EST_VC76', 'HC01_EST_VC77', 'HC03_EST_VC77',
                          'HC01_EST_VC78', 'HC03_EST_VC78', 'HC01_EST_VC79', 'HC03_EST_VC79'
                         ]].apply(pd.to_numeric, errors='coerce')
acs16_reduced['geo_id'] = acs16_df['GEO.id2']
acs16_reduced.rename(columns = {
    'HC01_EST_VC01': 'population_CNI',
    'HC03_EST_VC01': 'population_%insured',
    'HC01_EST_VC04': 'age_under18',
    'HC01_EST_VC07': 'age18to64',
    'HC03_EST_VC07': 'age18to64_%insured',
    'HC01_EST_VC08': 'age18to24',
    'HC03_EST_VC08': 'age18to24_%insured',
    'HC01_EST_VC09': 'age25to34',
    'HC03_EST_VC09': 'age25to34_%insured',
    'HC01_EST_VC10': 'age35to44',
    'HC03_EST_VC10': 'age35to44_%insured',
    'HC01_EST_VC11': 'age45to54',
    'HC03_EST_VC11': 'age45to54_%insured',
    'HC01_EST_VC12': 'age55to64',
    'HC03_EST_VC12': 'age55to64_%insured',
    'HC01_EST_VC13': 'age_over64',
    'HC01_EST_VC20': 'male',
    'HC03_EST_VC20': 'male_%insured',
    'HC01_EST_VC21': 'female',
    'HC03_EST_VC21': 'female_%insured',
    'HC01_EST_VC24': 'white',
    'HC03_EST_VC24': 'white_%insured',
    'HC01_EST_VC25': 'blackORaa',
    'HC03_EST_VC25': 'blackORaa_%insured',
    'HC01_EST_VC26': 'amerindian',
    'HC03_EST_VC26': 'amerindian_%insured',
    'HC01_EST_VC27': 'asian',
    'HC03_EST_VC27': 'asian_%insured',
    'HC01_EST_VC28': 'pIslander',
    'HC03_EST_VC28': 'pIslander_%insured',
    'HC01_EST_VC29': 'oRace',
    'HC03_EST_VC29': 'oRace_%insured',
    'HC01_EST_VC30': 'mixRace',
    'HC03_EST_VC30': 'mixRace_%insured',
    'HC01_EST_VC32': 'hispanicAlone',
    'HC03_EST_VC32': 'hispanicAlone_%insured',
    'HC01_EST_VC33': 'whiteAlone',
    'HC03_EST_VC33': 'whiteAlone_%insured',
    'HC01_EST_VC50': 'disability',
    'HC03_EST_VC50': 'disability_%insured',
    'HC01_EST_VC54': 'education_over25',
    'HC03_EST_VC54': 'education_over25_%insured',
    'HC01_EST_VC55': 'over25_lessThanHS',
    'HC03_EST_VC55': 'over25_lessThanHS_%insured',
    'HC01_EST_VC56': 'over25_HS',
    'HC03_EST_VC56': 'over25_HS_%insured',
    'HC01_EST_VC57': 'over25_someCollege',
    'HC03_EST_VC57': 'over25_someCollege_%insured',
    'HC01_EST_VC58': 'over25_college',
    'HC03_EST_VC58': 'over25_college_%insured',
    'HC01_EST_VC74': 'household_income',
    'HC03_EST_VC74': 'household_income_%insured',
    'HC01_EST_VC75': 'household_income_under$25K',
    'HC03_EST_VC75': 'household_income_under$25K_%insured',
    'HC01_EST_VC76': 'household_income_$25to49K',
    'HC03_EST_VC76': 'household_income_$25to49K_%insured',
    'HC01_EST_VC77': 'household_income_$50to74K',
    'HC03_EST_VC77': 'household_income_$50to74K_%insured',
    'HC01_EST_VC78': 'household_income_$75to99K',
    'HC03_EST_VC78': 'household_income_$75to99K_%insured',
    'HC01_EST_VC79': 'household_income_over$99K',
    'HC03_EST_VC79': 'household_income_over$99K_%insured'
}, inplace = True)

In [ ]:
acs16_reduced.dtypes

In [ ]:
acs16_reduced['percent_age18to24'] = acs16_reduced.age18to24 / acs16_reduced.age18to64
acs16_reduced['percent_age25to34'] = acs16_reduced.age25to34 / acs16_reduced.age18to64
acs16_reduced['percent_age35to44'] = acs16_reduced.age35to44 / acs16_reduced.age18to64
acs16_reduced['percent_age45to54'] = acs16_reduced.age45to54 / acs16_reduced.age18to64
acs16_reduced['percent_age55to64'] = acs16_reduced.age55to64 / acs16_reduced.age18to64
acs16_reduced['percent_male'] = acs16_reduced.male / acs16_reduced.population_CNI
acs16_reduced['percent_female'] = acs16_reduced.female / acs16_reduced.population_CNI
acs16_reduced['percent_hispanic'] = acs16_reduced.hispanicAlone / acs16_reduced.population_CNI
acs16_reduced['percent_white'] = acs16_reduced.whiteAlone / acs16_reduced.population_CNI
acs16_reduced['percent_disability'] = acs16_reduced.disability / acs16_reduced.population_CNI
acs16_reduced['percent_lessThanHS'] = acs16_reduced.over25_lessThanHS / acs16_reduced.education_over25
acs16_reduced['percent_HS'] = acs16_reduced.over25_HS / acs16_reduced.education_over25
acs16_reduced['percent_someCollege'] = acs16_reduced.over25_someCollege / acs16_reduced.education_over25
acs16_reduced['percent_college'] = acs16_reduced.over25_college / acs16_reduced.education_over25
acs16_reduced['percent_hIncome_under$25K'] = acs16_reduced['household_income_under$25K'] / acs16_reduced.household_income
acs16_reduced['percent_hIncome_$25to49K'] = acs16_reduced['household_income_$25to49K'] / acs16_reduced.household_income
acs16_reduced['percent_hIncome_$50to74K'] = acs16_reduced['household_income_$50to74K'] / acs16_reduced.household_income
acs16_reduced['percent_hIncome_$75to99K'] = acs16_reduced['household_income_$75to99K'] / acs16_reduced.household_income
acs16_reduced['percent_hIncome_over$99K'] = acs16_reduced['household_income_over$99K'] / acs16_reduced.household_income

In [ ]:
acs16_final = acs16_reduced[['population_CNI', 'age18to64', 'percent_age18to24', 'percent_age25to34',
                             'percent_age35to44', 'percent_age45to54', 'percent_age55to64', 'percent_male',
                             'percent_female', 'percent_hispanic', 'percent_white', 'percent_disability',
                             'percent_lessThanHS', 'percent_HS', 'percent_someCollege', 'percent_college',
                             'percent_hIncome_under$25K', 'percent_hIncome_$25to49K', 'percent_hIncome_$50to74K',
                             'percent_hIncome_$75to99K', 'percent_hIncome_over$99K',
                             'age18to64_%insured', 'male_%insured', 'female_%insured', 'hispanicAlone_%insured',
                             'whiteAlone_%insured', 'disability_%insured', 'over25_lessThanHS_%insured',
                             'over25_HS_%insured', 'over25_someCollege_%insured', 'over25_college_%insured',
                             'household_income_under$25K_%insured', 'household_income_$25to49K_%insured',
                             'household_income_$50to74K_%insured', 'household_income_$75to99K_%insured',
                             'household_income_over$99K_%insured', 'geo_id']]

## Aggregating by cities

In [ ]:
# Grouping by cities
tracts_groups = tracts_reduced.groupby(['geo_id'])

# Calculating mean, median and std for health indicators (HI)
HI_means = tracts_groups['HO_poor_mental_health', 'HO_poor_physical_health', 'UB_binge_drinking', 'UB_current_smoking',
                               'UB_lack_physical_activity', 'UB_obesity', 'UB_less_sleep', 'PS_lack_health_ins',
                              'PS_routine_checkups', 'PS_dental_checkups', 'PS_older_men_uptodate',
                               'PS_older_women_uptodate'].mean().reset_index()
HI_medians = tracts_groups['HO_poor_mental_health', 'HO_poor_physical_health', 'UB_binge_drinking', 'UB_current_smoking',
                               'UB_lack_physical_activity', 'UB_obesity', 'UB_less_sleep', 'PS_lack_health_ins',
                              'PS_routine_checkups', 'PS_dental_checkups', 'PS_older_men_uptodate',
                               'PS_older_women_uptodate'].median().reset_index()
HI_stdev = tracts_groups['HO_poor_mental_health', 'HO_poor_physical_health', 'UB_binge_drinking', 'UB_current_smoking',
                               'UB_lack_physical_activity', 'UB_obesity', 'UB_less_sleep', 'PS_lack_health_ins',
                              'PS_routine_checkups', 'PS_dental_checkups', 'PS_older_men_uptodate',
                               'PS_older_women_uptodate'].std().reset_index()

# Calculating counts of tracts per city
tracts_counts = tracts_groups['tract_FIPS'].count().reset_index()

## Merging datasets by geo_id

In [ ]:
# Create health indicators (HI) df
HI_df = pd.merge(HI_means, tracts_counts, on='geo_id')
print(f"There are {len(HI_df)} cities represented in the Health Indicators dataset.")
# Create master df
df = pd.merge(HI_df, acs16_final, on='geo_id')
print(f"There are {len(df)} cities represented in our final dataset, after merging with ACS_2016 dataset.")

In [ ]:
df.columns

## Binning by mental health (MH) scores

In [ ]:
df['HO_poor_mental_health'].describe()

In [ ]:
# Calculate quantiles on mental health outcome (MH) & prepare bins
bins = [5]
MH_Q1 = int(df['HO_poor_mental_health'].quantile(0.25))
bins.append(MH_Q1)
MH_Q2 = int(df['HO_poor_mental_health'].quantile(0.5))
bins.append(MH_Q2)
MH_Q3 = int(df['HO_poor_mental_health'].quantile(0.75))
bins.append(MH_Q3)
bins.append(20)
print(bins)
labels = ['Q1', 'Q2', 'Q3', 'Q4']
print(labels)

# Append our bins to df
df['quantile_MH'] = pd.cut(df['HO_poor_mental_health'], bins, labels=labels)
df_sorted = df.sort_values('quantile_MH')
df_sorted.head()

## Exploring differences in various health indicators by mental health (MH) bins

In [ ]:
df.boxplot('HO_poor_physical_health', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Poor Physical Health")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["HO_poor_physical_health"]
group2 = df[df["quantile_MH"] == 'Q2']["HO_poor_physical_health"]
group3 = df[df["quantile_MH"] == 'Q3']["HO_poor_physical_health"]
group4 = df[df["quantile_MH"] == 'Q4']["HO_poor_physical_health"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_PH = pairwise_tukeyhsd(df_sorted['HO_poor_physical_health'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_PH)

In [ ]:
df.boxplot('UB_binge_drinking', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Binge Drinking")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["UB_binge_drinking"]
group2 = df[df["quantile_MH"] == 'Q2']["UB_binge_drinking"]
group3 = df[df["quantile_MH"] == 'Q3']["UB_binge_drinking"]
group4 = df[df["quantile_MH"] == 'Q4']["UB_binge_drinking"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_UBbd = pairwise_tukeyhsd(df_sorted['UB_binge_drinking'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_UBbd)

In [ ]:
df.boxplot('UB_current_smoking', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Current Smoking")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["UB_current_smoking"]
group2 = df[df["quantile_MH"] == 'Q2']["UB_current_smoking"]
group3 = df[df["quantile_MH"] == 'Q3']["UB_current_smoking"]
group4 = df[df["quantile_MH"] == 'Q4']["UB_current_smoking"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_UBcs = pairwise_tukeyhsd(df_sorted['UB_current_smoking'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_UBcs)

In [ ]:
df.boxplot('UB_lack_physical_activity', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Lack of Physical Activity")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["UB_lack_physical_activity"]
group2 = df[df["quantile_MH"] == 'Q2']["UB_lack_physical_activity"]
group3 = df[df["quantile_MH"] == 'Q3']["UB_lack_physical_activity"]
group4 = df[df["quantile_MH"] == 'Q4']["UB_lack_physical_activity"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_UBpa = pairwise_tukeyhsd(df_sorted['UB_lack_physical_activity'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_UBpa)

In [ ]:
df.boxplot('UB_obesity', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Obesity")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["UB_obesity"]
group2 = df[df["quantile_MH"] == 'Q2']["UB_obesity"]
group3 = df[df["quantile_MH"] == 'Q3']["UB_obesity"]
group4 = df[df["quantile_MH"] == 'Q4']["UB_obesity"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_UBo = pairwise_tukeyhsd(df_sorted['UB_obesity'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_UBo)

In [ ]:
df.boxplot('UB_less_sleep', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Lack of Sleep")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["UB_less_sleep"]
group2 = df[df["quantile_MH"] == 'Q2']["UB_less_sleep"]
group3 = df[df["quantile_MH"] == 'Q3']["UB_less_sleep"]
group4 = df[df["quantile_MH"] == 'Q4']["UB_less_sleep"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_UBls = pairwise_tukeyhsd(df_sorted['UB_less_sleep'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_UBls)

In [ ]:
df.boxplot('PS_lack_health_ins', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Lack of Health Insurance")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["PS_lack_health_ins"]
group2 = df[df["quantile_MH"] == 'Q2']["PS_lack_health_ins"]
group3 = df[df["quantile_MH"] == 'Q3']["PS_lack_health_ins"]
group4 = df[df["quantile_MH"] == 'Q4']["PS_lack_health_ins"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_PShi = pairwise_tukeyhsd(df_sorted['PS_lack_health_ins'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_PShi)

In [ ]:
df.boxplot('PS_routine_checkups', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Routine Check-ups")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["PS_routine_checkups"]
group2 = df[df["quantile_MH"] == 'Q2']["PS_routine_checkups"]
group3 = df[df["quantile_MH"] == 'Q3']["PS_routine_checkups"]
group4 = df[df["quantile_MH"] == 'Q4']["PS_routine_checkups"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_PSrc = pairwise_tukeyhsd(df_sorted['PS_routine_checkups'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_PSrc)

In [ ]:
df.boxplot('PS_dental_checkups', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Dental Check-ups")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["PS_dental_checkups"]
group2 = df[df["quantile_MH"] == 'Q2']["PS_dental_checkups"]
group3 = df[df["quantile_MH"] == 'Q3']["PS_dental_checkups"]
group4 = df[df["quantile_MH"] == 'Q4']["PS_dental_checkups"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_PSdc = pairwise_tukeyhsd(df_sorted['PS_dental_checkups'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_PSdc)

In [ ]:
df.boxplot('PS_older_men_uptodate', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Older Men Up to Date with Clinical Preventive Services")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["PS_older_men_uptodate"]
group2 = df[df["quantile_MH"] == 'Q2']["PS_older_men_uptodate"]
group3 = df[df["quantile_MH"] == 'Q3']["PS_older_men_uptodate"]
group4 = df[df["quantile_MH"] == 'Q4']["PS_older_men_uptodate"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_PSom = pairwise_tukeyhsd(df_sorted['PS_older_men_uptodate'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_PSom)

In [ ]:
df.boxplot('PS_older_women_uptodate', by='quantile_MH', figsize=(20, 10))
plt.title("Prevalence of Older Women Up to Date with Clinical Preventive Services")
plt.show()

In [ ]:
# Extract individual groups
group1 = df[df["quantile_MH"] == 'Q1']["PS_older_women_uptodate"]
group2 = df[df["quantile_MH"] == 'Q2']["PS_older_women_uptodate"]
group3 = df[df["quantile_MH"] == 'Q3']["PS_older_women_uptodate"]
group4 = df[df["quantile_MH"] == 'Q4']["PS_older_women_uptodate"]

# Perform the ANOVA
stats.f_oneway(group1, group2, group3, group4)

In [ ]:
tukey_results_PSow = pairwise_tukeyhsd(df_sorted['PS_older_women_uptodate'], df_sorted['quantile_MH'], 0.05)
print(tukey_results_PSow)

## Exploring differences in demographic indicators by mental health (MH) bins

In [ ]:
df.columns

In [ ]:
# City population size and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['population_CNI'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("Mental Health vs City Population Size")
plt.xlabel("City Population Size")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['population_CNI'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['population_CNI'], p(df['population_CNI']), 'purple')
#plt.show()

# Box plot
df.boxplot('population_CNI', by='quantile_MH', figsize=(20, 10))
plt.title("Population Size by City")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["population_CNI"]
group2 = df[df["quantile_MH"] == 'Q2']["population_CNI"]
group3 = df[df["quantile_MH"] == 'Q3']["population_CNI"]
group4 = df[df["quantile_MH"] == 'Q4']["population_CNI"]
print(stats.f_oneway(group1, group2, group3, group4))

In [ ]:
# City population size (ages 18 to 64) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['age18to64'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("Mental Health vs City Population Size (ages 18 to 64)")
plt.xlabel("City Population Size (ages 18 to 64)")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['age18to64'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['age18to64'], p(df['age18to64']), 'purple')
#plt.show()

# Box plot
df.boxplot('age18to64', by='quantile_MH', figsize=(20, 10))
plt.title("City Population Size (ages 18 to 64 only)")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["age18to64"]
group2 = df[df["quantile_MH"] == 'Q2']["age18to64"]
group3 = df[df["quantile_MH"] == 'Q3']["age18to64"]
group4 = df[df["quantile_MH"] == 'Q4']["age18to64"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['age18to64'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# % population between ages 18 and 24 and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_age18to24'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% Population between ages 18 and 24 and prevalence of poor mental health")
plt.xlabel("% population between ages 18 and 24")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_age18to24'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_age18to24'], p(df['percent_age18to24']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_age18to24', by='quantile_MH', figsize=(20, 10))
plt.title("% population between ages 18 and 24")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_age18to24"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_age18to24"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_age18to24"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_age18to24"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_age18to24'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# % population between ages 25 and 34 and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_age25to34'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% Population between ages 25 and 34 and prevalence of poor mental health")
plt.xlabel("% population between ages 25 and 34")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_age25to34'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_age25to34'], p(df['percent_age25to34']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_age25to34', by='quantile_MH', figsize=(20, 10))
plt.title("% population between ages 25 and 34")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_age25to34"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_age25to34"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_age25to34"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_age25to34"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_age25to34'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# % population between ages 35 and 44 and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_age35to44'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% Population between ages 35 and 44 and prevalence of poor mental health")
plt.xlabel("% population between ages 35 and 44")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_age35to44'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_age35to44'], p(df['percent_age35to44']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_age35to44', by='quantile_MH', figsize=(20, 10))
plt.title("% population between ages 35 and 44")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_age35to44"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_age35to44"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_age35to44"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_age35to44"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_age35to44'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# % population between ages 45 and 54 and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_age45to54'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% Population between ages 45 and 54 and prevalence of poor mental health")
plt.xlabel("% population between ages 45 and 54")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_age45to54'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_age45to54'], p(df['percent_age45to54']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_age45to54', by='quantile_MH', figsize=(20, 10))
plt.title("% population between ages 45 and 54")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_age45to54"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_age45to54"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_age45to54"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_age45to54"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_age45to54'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# % population between ages 55 and 64 and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_age55to64'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% Population between ages 55 and 64 and prevalence of poor mental health")
plt.xlabel("% population between ages 55 and 64")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_age55to64'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_age55to64'], p(df['percent_age55to64']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_age55to64', by='quantile_MH', figsize=(20, 10))
plt.title("% population between ages 55 and 64")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_age55to64"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_age55to64"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_age55to64"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_age55to64"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_age55to64'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# % male and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_male'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% male in population and prevalence of poor mental health")
plt.xlabel("% male")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_male'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_male'], p(df['percent_male']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_male', by='quantile_MH', figsize=(20, 10))
plt.title("% male in population")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_male"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_male"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_male"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_male"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_male'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# % female and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_female'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% female in population and prevalence of poor mental health")
plt.xlabel("% female")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_female'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_female'], p(df['percent_female']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_female', by='quantile_MH', figsize=(20, 10))
plt.title("% female in population")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_female"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_female"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_female"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_female"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_female'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# % disability in population and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_disability'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% disability in population and prevalence of poor mental health")
plt.xlabel("% disability in population")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_disability'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_disability'], p(df['percent_disability']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_disability', by='quantile_MH', figsize=(20, 10))
plt.title("% disability in population")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_disability"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_disability"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_disability"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_disability"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_disability'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
df.columns

In [ ]:
# Educational attainment (less than a high school degree) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_lessThanHS'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% population with less than a high school degree and prevalence of poor mental health")
plt.xlabel("% population with less than a high school degreen")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_lessThanHS'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_lessThanHS'], p(df['percent_lessThanHS']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_lessThanHS', by='quantile_MH', figsize=(20, 10))
plt.title("% population with less than a high school degree")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_lessThanHS"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_lessThanHS"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_lessThanHS"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_lessThanHS"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_lessThanHS'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# Educational attainment (high school degree) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_HS'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% population with a high school degree and prevalence of poor mental health")
plt.xlabel("% population with a high school degree")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_HS'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_HS'], p(df['percent_HS']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_HS', by='quantile_MH', figsize=(20, 10))
plt.title("% population with a high school degree")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_HS"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_HS"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_HS"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_HS"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_HS'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# Educational attainment (some college or associate's degree) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_someCollege'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% population with some college or associate's degree and prevalence of poor mental health")
plt.xlabel("% population with some college or associate's degree")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_someCollege'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_someCollege'], p(df['percent_someCollege']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_someCollege', by='quantile_MH', figsize=(20, 10))
plt.title("% population with some college or associate's degree")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_someCollege"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_someCollege"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_someCollege"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_someCollege"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_someCollege'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# Educational attainment (Bachelor's degree or higher) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_college'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% population with a Bachelor's degree or higher and prevalence of poor mental health")
plt.xlabel("% population with a Bachelor's degree or higher")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_college'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_college'], p(df['percent_college']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_college', by='quantile_MH', figsize=(20, 10))
plt.title("% population with a Bachelor's degree or higher")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_college"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_college"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_college"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_college"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_college'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# Household income (under $25,000) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_hIncome_under$25K'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% population with household income under $25,000 and prevalence of poor mental health")
plt.xlabel("% population with household income under $25,000")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_hIncome_under$25K'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_hIncome_under$25K'], p(df['percent_hIncome_under$25K']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_hIncome_under$25K', by='quantile_MH', figsize=(20, 10))
plt.title("% population with household income under $25,000")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_hIncome_under$25K"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_hIncome_under$25K"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_hIncome_under$25K"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_hIncome_under$25K"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_hIncome_under$25K'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# Household income ($25,000-$49,999) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_hIncome_$25to49K'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% population with household income between $25,000 and $49,999 and prevalence of poor mental health")
plt.xlabel("% population with household income between $25,000 and $49,999")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_hIncome_$25to49K'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_hIncome_$25to49K'], p(df['percent_hIncome_$25to49K']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_hIncome_$25to49K', by='quantile_MH', figsize=(20, 10))
plt.title("% population with household income between $25,000 and $49,999")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_hIncome_$25to49K"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_hIncome_$25to49K"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_hIncome_$25to49K"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_hIncome_$25to49K"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_hIncome_$25to49K'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# Household income ($50,000-$74,999) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_hIncome_$50to74K'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% population with household income between $50,000 and $74,999 and prevalence of poor mental health")
plt.xlabel("% population with household income between $50,000 and $74,999")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_hIncome_$50to74K'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_hIncome_$50to74K'], p(df['percent_hIncome_$50to74K']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_hIncome_$50to74K', by='quantile_MH', figsize=(20, 10))
plt.title("% population with household income between $50,000 and $74,999")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_hIncome_$50to74K"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_hIncome_$50to74K"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_hIncome_$50to74K"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_hIncome_$50to74K"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_hIncome_$50to74K'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# Household income ($75,000-$99,999) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_hIncome_$75to99K'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% population with household income between $75,000 and $99,999 and prevalence of poor mental health")
plt.xlabel("% population with household income between $75,000 and $99,999")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_hIncome_$75to99K'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_hIncome_$75to99K'], p(df['percent_hIncome_$75to99K']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_hIncome_$75to99K', by='quantile_MH', figsize=(20, 10))
plt.title("% population with household income between $75,000 and $99,999")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_hIncome_$75to99K"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_hIncome_$75to99K"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_hIncome_$75to99K"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_hIncome_$75to99K"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_hIncome_$75to99K'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
# Household income (over $99,999) and prevalence of poor mental health

# Scatter plot:
plt.scatter(df['percent_hIncome_over$99K'], df['HO_poor_mental_health'], marker='o', facecolors='blue',
            edgecolors='black', alpha=0.5)
plt.title("% population with household income over $99,999 and prevalence of poor mental health")
plt.xlabel("% population with household income over $99,999")
plt.ylabel("Prevalence of Poor Mental Health")
# adding a trendline
z = np.polyfit(df['percent_hIncome_over$99K'], df['HO_poor_mental_health'], 1)
p = np.poly1d(z)
plt.plot(df['percent_hIncome_over$99K'], p(df['percent_hIncome_over$99K']), 'purple')
#plt.show()

# Box plot
df.boxplot('percent_hIncome_over$99K', by='quantile_MH', figsize=(20, 10))
plt.title("% population with household income over $99,999")
#plt.show()

# Perform the ANOVA
group1 = df[df["quantile_MH"] == 'Q1']["percent_hIncome_over$99K"]
group2 = df[df["quantile_MH"] == 'Q2']["percent_hIncome_over$99K"]
group3 = df[df["quantile_MH"] == 'Q3']["percent_hIncome_over$99K"]
group4 = df[df["quantile_MH"] == 'Q4']["percent_hIncome_over$99K"]
print(stats.f_oneway(group1, group2, group3, group4))

# Perform Tukey's test
tukey_results = pairwise_tukeyhsd(df_sorted['percent_hIncome_over$99K'], df_sorted['quantile_MH'], 0.05)
print(tukey_results)

In [ ]:
df.columns